# Netmiko & Diffing

## Prepare Your Environment

Spin up a new virtual environment using your tool of choice (`virtualenv`, `virtualenvwrapper`, etc.) and install a number of libraries:

```
pip install netmiko ncclient jupyter
```

Then run up `jupyter-notebook` and open this file.

## Connection Variables

In [16]:
# Local confd
HOST = '127.0.0.1'
PORT_NC = 2022
PORT_SSH = 2024
USER = 'admin'
PASS = 'admin'

# used for netmiko device connection
PLATFORM = 'cisco_ios'

## Connect both netmiko and ncclient

In [17]:
from netmiko import ConnectHandler
from ncclient import manager
from lxml import etree


def pretty_print(retval):
    print(etree.tostring(retval.data, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True

def get_reply(chan, eom):
    bytes = u''
    while bytes.find(eom)==-1:
        bytes += chan.recv(65535).decode('utf-8')
    return bytes

m = manager.connect(host=HOST, port=PORT_NC, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)
d = ConnectHandler(device_type=PLATFORM, ip=HOST, port=PORT_SSH, username=USER, password=PASS)

prompt = d.find_prompt()

## Sample Config To Apply

In [18]:
config_old = '''ip access-list standard FOO
 permit 10.128.0.0 0.127.255.255
 permit 10.119.120.0 0.0.7.255
 permit 10.87.79.0 0.0.0.255
 permit 10.87.96.0 0.0.0.255
 deny   any
'''

# ConfD ACLS **require** sequence number and a commit
config = '''ip access-list standard FOO
 10 permit 10.128.0.0 0.127.255.255
 20 permit 10.119.120.0 0.0.7.255
 30 permit 10.87.79.0 0.0.0.255
 40 permit 10.87.96.0 0.0.0.255
 50 deny   any
commit
'''

## Apply Config

In [19]:
# need to strip leading whitespace for confd, or it gets confused
input = [i.strip() for i in config.splitlines()]

# apply config
output = d.send_config_set(config.splitlines())

## Capture and Display Netconf Config

In [20]:
retval = m.get_config(source='running', filter=('xpath', '/native'))
native = retval.data_ele[0]

In [21]:
after_xml_str = etree.tostring(native, pretty_print=True)
print(after_xml_str.decode())

<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <ip>
    <access-list>
      <standard xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-acl">
        <name>FOO</name>
        <access-list-seq-rule>
          <sequence>10</sequence>
          <permit>
            <std-ace>
              <ipv4-prefix>10.128.0.0</ipv4-prefix>
              <mask>0.127.255.255</mask>
            </std-ace>
          </permit>
        </access-list-seq-rule>
        <access-list-seq-rule>
          <sequence>20</sequence>
          <permit>
            <std-ace>
              <ipv4-prefix>10.119.120.0</ipv4-prefix>
              <mask>0.0.7.255</mask>
            </std-ace>
          </permit>
        </access-list-seq-rule>
        <access-list-seq-rule>
          <sequence>30</sequence>
          <permit>
            <std-ace>
              <ipv4-prefix>10.87.79.0</ipv4-prefix>
              <mask>0.0.0.255</mask>
            </std-ace>
      

## Delete ALL Native Config

In [14]:
delete_native = '''
<config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native" nc:operation="delete"/>
</config>
'''

retval= m.edit_config(delete_native, format='xml', target='running')

TransportError: Not connected to NETCONF server

## Tidyup Sessions

In [15]:
d.disconnect()
m.close_session()

TransportError: Not connected to NETCONF server